This is a Data Exploration on the asian beauty subreddit.  I will be exploring the posts asking for recommendations for products for specific skin types and exploring which products are the most recommended in the comments/replies section. 

I will specifically be looking at a few threads where the comments are structured since it is difficult to extract data from unstructured posts.  A future next step is to look into phrase mining research done by the University of Illinois or named entity recongition with a Stanford package to look at unstructured posts and significantly expand the quantity of data. 

Import the necessary packages.  I initially had trouble importing praw because the notebook was running a different Python than the one in my command prompt so it could not find the import.  I fixed it by checking sys.executable and sys.path to see where the notebook is looking for imports and did path -m pip install praw.

In [ ]:
#! usr/bin/env python3
import sys
import praw
import numpy as np
import pandas as pd 
import string
from datetime import datetime
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sys.executable


Set up the Reddit API. I am looking at /r/AsianBeauty 

In [ ]:
reddit = praw.Reddit(client_id='14_CHARS_IDENTIFIER', 
                     client_secret='27_CHARS_SECRET_ID', 
                     user_agent='SCRIPT_NAME', 
                     username='REDDIT_USER_NAME', 
                     password='REDDIT_LOGIN_PASSWORD')

asian_beauty=reddit.subreddit('AsianBeauty')

Collect the Data from a few posts where the comments are structured. 

In [ ]:
dict = {"id" : [],
        "product" : [], 
        "step" : [],
        "opinion" : [],
        "thoughts" : [],
        "thread": []
}

submission = reddit.submission(id = '6lmzyl')

for comment in submission.comments:
    if(len(comment.body.split("\n\n")) >= 4 and comment.body[0] == "*"):
        dict["id"].append(comment.id)
        dict["product"].append(comment.body.split("\n\n")[0].replace("**",""))
        dict["step"].append(comment.body.split("\n\n")[1].replace("**",""))
        dict["opinion"].append(comment.body.split("\n\n")[2].replace("**",""))
        dict["thoughts"].append(comment.body.split("\n\n")[3].replace("**",""))  
        dict["thread"].append("Oily")
       
submission = reddit.submission(id = '6na9hw')
for comment in submission.comments:
    if(len(comment.body.split("\n\n")) >= 4):
        dict["id"].append(comment.id)
        dict["product"].append(comment.body.split("\n\n")[0].replace("**",""))
        dict["step"].append(comment.body.split("\n\n")[1].replace("**",""))
        dict["opinion"].append(comment.body.split("\n\n")[2].replace("**",""))
        dict["thoughts"].append(comment.body.split("\n\n")[3].replace("**",""))  
        dict["thread"].append("Dry")
        
submission = reddit.submission(id = '6uijtv')
for comment in submission.comments:
    if(len(comment.body.split("\n\n")) >= 4):
        dict["id"].append(comment.id)
        dict["product"].append(comment.body.split("\n\n")[0].replace("**",""))
        dict["step"].append(comment.body.split("\n\n")[1].replace("**",""))
        dict["opinion"].append(comment.body.split("\n\n")[2].replace("**",""))
        dict["thoughts"].append(comment.body.split("\n\n")[3].replace("**",""))  
        dict["thread"].append("Combination")
        
submission = reddit.submission(id = '6remns')
for comment in submission.comments:
    if(len(comment.body.split("\n\n")) >= 4):
        dict["id"].append(comment.id)
        dict["product"].append(comment.body.split("\n\n")[0].replace("**",""))
        dict["step"].append(comment.body.split("\n\n")[1].replace("**",""))
        dict["opinion"].append(comment.body.split("\n\n")[2].replace("**",""))
        dict["thoughts"].append(comment.body.split("\n\n")[3].replace("**",""))  
        dict["thread"].append("Acne")
        
data = pd.DataFrame(dict)
data

Experiemnting on how to get a comment forest (the replies to a comment).  Since there are only 54 items in the list and I want to be able to quantify the popularity of a product by applying sentiment analysis on the comments to see if it's a positive comment or negative. 


In [ ]:
sid = SentimentIntensityAnalyzer()
for comment in submission.comments:
    comment.refresh()
    for reply in comment.replies:
        print(reply.body)
        ss = sid.polarity_scores(reply.body)
        for k in ss:
            print( "{0}: {1}," .format(k, ss[k]), end=" ")
            print()

So the sentiment analyzer seems to work quite well! I'm going to add a positive count and negative count column to the data set to record the comments. 

In [ ]:
sid = SentimentIntensityAnalyzer()
data["pos_count"] = 0
data["neg_count"] = 0
data["neu_count"] = 0
count_row = len(data)

for i in range(0,count_row): 
    comment = reddit.comment(data['id'].iloc[i])
    comment.refresh()
    #print(comment.body)
    for reply in comment.replies:
        #print(reply.body)
        ss = sid.polarity_scores(reply.body)
        
        if ss["compound"] == 0.0:
            #print("neutral")
            data.loc[i, "neu_count" ] += 1
        elif ss["compound"] > 0.0:
            #print("postive")
            data.loc[i, "pos_count" ] += 1
        else:
            #print("neg")
            data.loc[i, "neg_count" ] += 1
        

In [ ]:
data.head()

In [ ]:
data = data.apply(lambda x: x.str.lower() if(x.dtype == 'object') else x)

data.loc[data['opinion'].str.contains("best"), 'pos_count'] = data['pos_count'] + 1
data.loc[data['opinion'].str.contains("worst"), 'pos_count'] = data['pos_count'] + 1


data

In [ ]:
data.to_csv("skincare.csv", encoding='utf-8', index=False)